In [1]:
from GoogleAIClient import GoogleAIClient

In [3]:
api_key =''

client = GoogleAIClient(
    api_key=api_key
)

In [10]:
model= 'gemini-1.5-flash'

In [5]:
from datasets import load_from_disk

mmlu   = load_from_disk(r"/Users/sasha/Desktop/hugging_face/metrics/datasets/mmlu")
const  = load_from_disk(r"/Users/sasha/Desktop/hugging_face/metrics/datasets/const")
dastur = load_from_disk(r"/Users/sasha/Desktop/hugging_face/metrics/datasets/dastur")
ent    = load_from_disk(r"/Users/sasha/Desktop/hugging_face/metrics/datasets/ent")

/Users/sasha/Desktop/hugging_face/metrics/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import json

context_remove = set()
def format_mmlu_like(entry, dataset_name, index):
    title = entry.get("Title", "")
    question = entry.get("Question", "")
    text = entry.get("Context")
    if text is None:
        context_remove.add(dataset_name.split('-')[0])
    options = []
    for opt_key in ["Option A", "Option B", "Option C", "Option D"]:
        if opt_key in entry and entry[opt_key] is not None:
            options.append(entry[opt_key])
            
    user_prompt = f"Title: {title}\nQuestion: {question}\nOptions:\n"

    for i, opt in enumerate(options):
        label = chr(ord('A') + i)
        user_prompt += f"{label}: {opt}\n"
        
    user_prompt += "\nСұраққа жауапты тек бір ғана әріппен (A, B, C, т.б.) дереу көрсетіңіз. Сұраққа жауап ретінде тек бір ғана әріпті (мысалы, A, B, C, т.б.) көрсету керек. Егер модель дұрыс жауап жоқ деп ойласа немесе бірнеше дұрыс жауап бар деп есептесе де, бәрібір тек бір ғана әріпті таңдап, оны жауаптың басында бірден басып шығару қажет. Басқа ешқандай мәтін немесе түсіндірме берілмейді, тек таңдалған әріп. Бір ғана дұрыс жауаптың әріпін көрсетіңіз:"
    
    json_line = {
        "custom_id": f"{dataset_name}-{index}",
        "content": f"Сіз қазақ тілінде жауап беретін білімді, пайдалы көмекшісіз. Қазақстан мәдениеті, ғылым, тарих және басқа да салалар бойынша көптаңдаулы сұрақтарға жауап беріңіз. Сұрақты және берілген жауап нұсқаларын мұқият оқып, ең дұрысын бір ғана әріппен (A, B, C, т.б.) белгілеңіз.\n\n{user_prompt}"
        }

    return json_line


def format_ent_like(entry, dataset_name, subject_name, index):
    subject = entry.get("subject", "")
    question = entry.get("question", "")
    
    option_keys = [k for k in ['A','B','C','D','E','F','G','H'] if k in entry and entry[k] is not None and entry[k] != '<missing>']
    
    user_prompt = f"Subject: {subject}\nQuestion: {question}\nOptions:\n"
    for key in option_keys:
        user_prompt += f"{key}: {entry[key]}\n"
    user_prompt += "\nСұраққа жауапты тек бір ғана әріппен (A, B, C, т.б.). Сұраққа жауап ретінде тек бір ғана әріпті (мысалы, A, B, C, т.б.) көрсету керек. Егер модель дұрыс жауап жоқ деп ойласа немесе бірнеше дұрыс жауап бар деп есептесе де, бәрібір тек бір ғана әріпті таңдап, оны жауаптың басында бірден басып шығару қажет. Басқа ешқандай мәтін немесе түсіндірме берілмейді, тек таңдалған әріп. Бір ғана дұрыс жауаптың әріпін көрсетіңіз:"
    
    json_line = {
        "custom_id": f"{dataset_name}-{subject_name}-{index}",
        "content": f"Сіз қазақ тілінде жауап беретін білімді, пайдалы көмекшісіз. Қазақстан мәдениеті, ғылым, тарих және басқа да салалар бойынша көптаңдаулы сұрақтарға жауап беріңіз. Сұрақты және берілген жауап нұсқаларын мұқият оқып, ең дұрысын бір ғана әріппен (A, B, C, т.б.) белгілеңіз.\n\n{user_prompt}"
        }

    return json_line


with open("batch_requests_flash.jsonl", "w", encoding="utf-8") as f:

    ds = mmlu["validation"]
    for i, entry in enumerate(ds):
        line = format_mmlu_like(entry, "mmlu", i)
        f.write(json.dumps(line, ensure_ascii=False) + "\n")

    for dataset_name, dataset_obj in [("const", const), ("dastur", dastur)]:
        ds = dataset_obj["test"]
        for i, entry in enumerate(ds):
            line = format_mmlu_like(entry, dataset_name, i)
            f.write(json.dumps(line, ensure_ascii=False) + "\n")

    for subject_name in ent.keys():
        ds = ent[subject_name]
        for i, entry in enumerate(ds):
            line = format_ent_like(entry, "ent", subject_name, i)
            f.write(json.dumps(line, ensure_ascii=False) + "\n")


In [7]:
import json

file_path = r"batch_requests_flash.jsonl"

data = []
with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        data.append(json.loads(line))

print(data[0])

{'custom_id': 'mmlu-0', 'content': 'Сіз қазақ тілінде жауап беретін білімді, пайдалы көмекшісіз. Қазақстан мәдениеті, ғылым, тарих және басқа да салалар бойынша көптаңдаулы сұрақтарға жауап беріңіз. Сұрақты және берілген жауап нұсқаларын мұқият оқып, ең дұрысын бір ғана әріппен (A, B, C, т.б.) белгілеңіз.\n\nTitle: abstract_algebra\nQuestion: 18 арқылы жасалған Z_24 циклдік топшасының реті бар\nOptions:\nA: 4\nB: 8\nC: 12\nD: 6\n\nСұраққа жауапты тек бір ғана әріппен (A, B, C, т.б.) дереу көрсетіңіз. Сұраққа жауап ретінде тек бір ғана әріпті (мысалы, A, B, C, т.б.) көрсету керек. Егер модель дұрыс жауап жоқ деп ойласа немесе бірнеше дұрыс жауап бар деп есептесе де, бәрібір тек бір ғана әріпті таңдап, оны жауаптың басында бірден басып шығару қажет. Басқа ешқандай мәтін немесе түсіндірме берілмейді, тек таңдалған әріп. Бір ғана дұрыс жауаптың әріпін көрсетіңіз:'}


In [11]:
len(data)

17800

In [16]:
import time
from tqdm import tqdm

results = []
max_retries = 3

for i in tqdm(range(len(data))):
# for i in tqdm(range(20)):
    current_attempt = 0
    success = False

    while current_attempt < max_retries:
        try:
            id = data[i]['custom_id']
            prompt = data[i]['content']

            ans = client.completions_request(max_tokens=20, prompt=prompt, model=model)

            results.append({'custom_id': id, 'answer': ans['text']})
            success = True
            break
        except Exception as e:
            print(f"Attempt {current_attempt + 1} failed for index {i}: {e}")
            current_attempt += 1
            time.sleep(10)

    if not success:
        results.append({'custom_id': id, 'answer': None})
        print(f"All attempts failed for index {i}, appending None.")

 16%|█▋        | 2926/17800 [22:53<1:42:24,  2.42it/s]

Attempt 1 failed for index 2926: HTTP error: The read operation timed out


 36%|███▌      | 6389/17800 [50:26<1:26:01,  2.21it/s] 

Attempt 1 failed for index 6389: HTTP error: The read operation timed out


100%|██████████| 17800/17800 [2:16:23<00:00,  2.17it/s]  


In [18]:
import numpy as np

np.save(arr=results, file='gemini_15_flash.npy')

In [17]:
results

[{'custom_id': 'mmlu-0', 'answer': 'A\n'},
 {'custom_id': 'mmlu-1', 'answer': 'A\n'},
 {'custom_id': 'mmlu-2', 'answer': 'C\n'},
 {'custom_id': 'mmlu-3', 'answer': 'B\n'},
 {'custom_id': 'mmlu-4', 'answer': 'C\n'},
 {'custom_id': 'mmlu-5', 'answer': 'D\n'},
 {'custom_id': 'mmlu-6', 'answer': 'C\n'},
 {'custom_id': 'mmlu-7', 'answer': 'D\n'},
 {'custom_id': 'mmlu-8', 'answer': 'C\n'},
 {'custom_id': 'mmlu-9', 'answer': 'D\n'},
 {'custom_id': 'mmlu-10', 'answer': 'C\n'},
 {'custom_id': 'mmlu-11', 'answer': 'D\n'},
 {'custom_id': 'mmlu-12', 'answer': 'D\n'},
 {'custom_id': 'mmlu-13', 'answer': 'C\n'},
 {'custom_id': 'mmlu-14', 'answer': 'C\n'},
 {'custom_id': 'mmlu-15', 'answer': 'B\n'},
 {'custom_id': 'mmlu-16', 'answer': 'C\n'},
 {'custom_id': 'mmlu-17', 'answer': 'A\n'},
 {'custom_id': 'mmlu-18', 'answer': 'A\n'},
 {'custom_id': 'mmlu-19', 'answer': 'C\n'},
 {'custom_id': 'mmlu-20', 'answer': 'B\n'},
 {'custom_id': 'mmlu-21', 'answer': 'B\n'},
 {'custom_id': 'mmlu-22', 'answer': 'B\n'}